<a href="https://colab.research.google.com/github/mesahwi/TextAnlaysis/blob/master/NewsCrawler_Naver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple tutorial of downloading news from naver.com, to google drive

Install chrome driver and selenium

In [1]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (75.0.3770.90-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [3]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [4]:
!pip install selenium

Mount Google Drive

In [5]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
import urllib.request
from selenium import webdriver
import os, time
import numpy as np
from datetime import timedelta, date

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [0]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

Get a list of the user specified articles

In [8]:
base_dir = 'gdrive/Shared drives/텍스트마이닝/News Analysis/'

start_date = date(2019,1,8)
end_date = date(2019,7,8)

paper = 'Chosun'
section = 'tech'

savePath = base_dir + paper + '/' + section + '/'

if paper == 'Chosun':
  oid = '023'
elif paper == 'Donga':
  oid = '020'
elif paper == 'Han':
  oid = '028'
elif paper == 'Kyung':
  oid = '032'
elif paper == 'Chung':
  oid = '025'
elif paper == 'Presian':
  oid = '002'

if section == 'politics':
  sid1 = '100'
elif section == 'tech':
  sid1 = '105'
elif section == 'society':
  sid1 = '103'
  

list_baseurl = 'https://news.naver.com/main/list.nhn?mode=LPOD&sid1=' + sid1+ '&mid=shm&oid=' + oid + '&listType=title&date='
list_hrefList = []

directory = os.getcwd()
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(1)

print('opening selenium...')
for single_date in daterange(start_date, end_date):
  search_date = single_date.strftime('%Y%m%d')
  driver.get(list_baseurl+search_date)
  elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul[*]/li[*]/a')
  for i, element in enumerate(elements):
    if i > 5 : break  # max 4 articles per day
    list_hrefList.append(element.get_attribute('href'))
    
    
hrefList = np.unique(list_hrefList)

opening selenium...


In [9]:
#check save path and the first 5
print(savePath)
print(hrefList[:5])
print(len(hrefList))

gdrive/Shared drives/텍스트마이닝/News Analysis/Chosun/tech/
['https://news.naver.com/main/read.nhn?mode=LPOD&mid=shm&sid1=105&oid=023&aid=0003420242'
 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=shm&sid1=105&oid=023&aid=0003420243'
 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=shm&sid1=105&oid=023&aid=0003420244'
 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=shm&sid1=105&oid=023&aid=0003420415'
 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=shm&sid1=105&oid=023&aid=0003420429']
592


Download the articles in the list

In [10]:
#downlad articles! 
article_xpath = '//*[@id="articleBodyContents"]'

for i, href in enumerate(hrefList):
  if(i%50 == 0):
    print(str(i) + 'th article downloading...')
  driver.get(href)
  article = driver.find_elements_by_xpath(article_xpath)
  if len(article) > 0 :
    f = open(savePath+href[-10:]+'.txt', 'w')
    f.write(article[0].text)
    f.close()
  else : 
    print('invalid url will be ignored... : ', href)

0th article downloading...
50th article downloading...
100th article downloading...
150th article downloading...
200th article downloading...
250th article downloading...
300th article downloading...
350th article downloading...
400th article downloading...
450th article downloading...
500th article downloading...
550th article downloading...


Check how many articles were actually downloaded

In [11]:
ls 'gdrive/Shared drives/텍스트마이닝/News Analysis/Chosun/tech'| wc -l

592
